In [1]:
!pip install transformers

     |████████████████████████████████| 2.5MB 8.7MB/s 
     |████████████████████████████████| 901kB 45.9MB/s 
     |████████████████████████████████| 3.3MB 49.3MB/s 


In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import nltk
import numpy as np
import pandas as pd

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id = tokenizer.eos_token_id)

In [4]:
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu

def bleu(ref, gen):
    ''' 
    calculate pair wise bleu score. uses nltk implementation
    Args:
        references : a list of reference sentences 
        candidates : a list of candidate(generated) sentences
    Returns:
        bleu score(float)
    '''
    ref_bleu = []
    gen_bleu = []
    for l in gen:
        gen_bleu.append(l.split())
    for i,l in enumerate(ref):
        ref_bleu.append([l.split()])
    cc = SmoothingFunction()
    score_bleu = corpus_bleu(ref_bleu, gen_bleu, weights=(0, 1, 0, 0), smoothing_function=cc.method4)
    return score_bleu

In [5]:
df = pd.read_excel('Data_SOC.xlsx')
df = pd.DataFrame(df)
df1 = pd.DataFrame(columns={'Finance','Sc_Literature','Music/Entertainment'})
df1.head()

,Finance,Music/Entertainment,Sc_Literature


In [7]:
i = 0
while( i < len(df)):
  content = df['Sc_Literature'][i]
  input_ids = tokenizer.encode(content[:100], return_tensors='pt')
  output = model.generate(input_ids, max_length = 1000, do_sample = True, top_k = 50, top_p = 0.95)

  reference = tokenizer.decode(output[0], skip_special_tokens= True).split()

  len1 = len(content)
  len2 = len(reference)
  if(len1 >= len2):
    hypothesis = content.split()[:len2]
  else:
    reference = reference[:len1]
    hypothesis = content.split()

  BLEUscore = nltk.translate.bleu_score.corpus_bleu([reference], [hypothesis])
  df1.loc[i,'Sc_Literature'] = BLEUscore
  print(BLEUscore)
  i += 1

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.3116545944402656
0.3386854985606571
0.3406637207527856
0
0.3429971702850177
0.33785504155934964
0.2874356249270482
0.3381307292971254
0.33256616292381014
0.30901626512512986


In [8]:
i = 0
while( i != len(df)):
  content = df['Finance'][i]
  input_ids = tokenizer.encode(content[:100], return_tensors='pt')
  output = model.generate(input_ids, max_length = 1000, do_sample = True, top_k = 50, top_p = 0.95)

  reference = tokenizer.decode(output[0], skip_special_tokens= True).split()
  
  len1 = len(content)
  len2 = len(reference)
  if(len1 >= len2):
    hypothesis = content.split()[:len2]
  else:
    reference = reference[:len1]
    hypothesis = content.split()

  BLEUscore = nltk.translate.bleu_score.corpus_bleu([reference], [hypothesis])
  df1.loc[i,'Finance'] = BLEUscore
  print(BLEUscore)
  i += 1

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.31809980918452735
0.27474558342153516
0.3078532175257951
0.31214655271081343
0.27986790633939385
0.3839817133079349
0.3341094348221695
0.27922771631509835
0.30536941697562214
0.3517355751503512


In [9]:
i = 0
while( i != len(df)):
  content = df['Music/Entertainment'][i]
  input_ids = tokenizer.encode(content[:100], return_tensors='pt')
  output = model.generate(input_ids, max_length = 1000, do_sample = True, top_k = 50, top_p = 0.95)

  reference = tokenizer.decode(output[0], skip_special_tokens= True).split()

  len1 = len(content)
  len2 = len(reference)
  if(len1 >= len2):
    hypothesis = content.split()[:len2]
  else:
    reference = reference[:len1]
    hypothesis = content.split()

  BLEUscore = nltk.translate.bleu_score.corpus_bleu([reference], [hypothesis])
  df1.loc[i,'Music/Entertainment'] = BLEUscore
  print(BLEUscore)
  i += 1

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.3976353643835253
0.34329452398451965
0.40276720463657734
0.33980884896942454
0.2773500981126146
0.30901626512512986
0.3023477224991132
0
0.346966645494591
0.2753358028987337


In [11]:
df1.to_csv(r'BLEU_Scores.csv', index=False)